Vraagjes aan de docenten:
- Wat wordt er precies bedoeld met 6 uur "block time"?
- 58 minuten waar ronden we naar af?
- How to deduct satisfied demand after an aircraft route is saved when demand is also captured from the previous two bins (does the demand gets lower for each bin according to their composition in the total flow in a bin)?
- What information should be included in the state? (place? time? profit at that time? aircraft?)


In [4]:
# Group: 28
# Authors: Thieme Brandwagt and Sam Buisman
#=========================================
# Assignment 2 - Aircraft Routing Problem 
#=========================================

# Import necessary packages
from openpyxl import *
from time import *
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# profit, n, aircraft, i, j = s # Attributes for state


### STEP 1: Load the data

In [6]:
# Load airport data into a numpy array
file_path_airports = "DATA/AirportData.xlsx"
airports_df = pd.read_excel(file_path_airports, usecols ="B:U")
airports = airports_df.to_numpy()
print(airports[0][0])
# Create list for all the airports (columns)
cities = airports_df.columns
print(cities[0])


# Load fleet data into a numpy array
file_path_fleet = "DATA/FleetType.xlsx"
fleet_df = pd.read_excel(file_path_fleet)
fleet_df = fleet_df.set_index("Aircraft Type")
fleet = fleet_df.to_numpy()
print(fleet[0][0])
# Create list for all the aircraft types (columns)
aircraft_types = fleet_df.columns
print(aircraft_types[0])


# Load demand data into a numpy array
file_path_demand = "DATA/Group28.xlsx"
demand_df = pd.read_excel(file_path_demand, usecols="B:AG", skiprows=4)
# days = 5
# column_labels = []
# column_labels.extend([f"Day {day+1} {interval}" for day in range(days) for interval in time_intervals])
# demand_df.columns = column_labels
demand = demand_df.to_numpy()
print(demand[0][0])

# Create a triple dimensional array for demand data (origin, destination, time)
demand_matrix = np.zeros((len(cities), len(cities), demand.shape[1] - 2))
# Populate matrix with all demand from all intervals
for rows in range(len(cities)):
    for columns in range(len(cities)):
        for time_steps in range(int(120/6)):  # 120 hours devided by 4 hour time steps
            demand_matrix[rows][columns][time_steps] = demand[rows * 20 + columns][time_steps + 2]
print(demand_matrix[19][18][6])


LHR
London
800.0
Type 1: Small freighter
LHR
11396.314893372413


In [7]:
time_intervals = ["00:00-04:00", "04:00-08:00", "08:00-12:00", 
                  "12:00-16:00", "16:00-20:00", "20:00-00:00"]

In [8]:
tijd = [0,1,2,3,4,5]
tijd[-1]

5

### STEP 2: Define the functions

In [9]:
# Adding the previous two bins to the currect bin to get total demand
def flow(i, j, t):
    return demand_matrix[i,j,t] + 0.2 * (demand_matrix[i,j,t-1]+demand_matrix[i,j,t-2])

flow(1,2,8)

118576.59293003316

In [10]:
def distance(i, j, R_E=6371):
    if i != j:
        lat_i, long_i = airports[1][i], airports[2][i]
        lat_j, long_j = airports[1][j], airports[2][j]
        delta_lat = lat_j - lat_i
        delta_long = long_j - long_i
        root = math.sin(math.radians(delta_lat)/2)**2 + math.cos(math.radians(lat_i)) \
            * math.cos(math.radians(lat_j)) * math.sin(math.radians(delta_long)/2)**2
        return R_E * 2 * math.asin(math.sqrt(root)) 
    else:
        return 0

# E.v.t. nog stoppen in een df'pje later, zodat ie niet naar zichzelf gaat verwijzen

In [34]:
time_step = 6
total_time = 120 * 60
stages = np.arange(0, total_time + 1, time_step)
print(stages)

stages[5]

def flight_duration(i,j,k):
    return distance(i, j, R_E=6371) / fleet[0][k]

# SAM KLOPT DIE +1 WANT HET GETAL IS EEN BEETJE RAAR
time_stages = np.arange(0, total_time + 1, time_step)
time_bins = np.arange(30)

divider = len(time_stages)/ len(time_bins) # Divide by 40 to get the right bin associated with the stage

print(divider)

[   0    6   12 ... 7188 7194 7200]
40.03333333333333


In [40]:
# Calculate the cost for each flight leg
def cost_flight_leg(i, j, k):
    if i != j:
        tc = fleet[7][k] * (distance(i, j, R_E=6371) / fleet[0][k])     # formula for the time costs
        fc = ((fleet[8][k] * 1.42) / 1.5) * distance(i, j, R_E=6371)    # formula for the fuel costs
        foc = fleet[6][k]                                               # formula for the fixed operating costs
        return tc + fc + foc

print(cost_flight_leg(0,2,2))

# Calculate the profit for each flight leg
def profit(i, j, k, s):
    satisfied_demand = min(flow(i,j,math.floor((s / divider))), fleet[1][k])
    if i != j:
        revenue = 0.26 * distance(i,j) * satisfied_demand / 1000
        costs = cost_flight_leg(i,j,k)
        return revenue - costs

print(profit(0, 2, 2, 0))

# Calculate the flight duration for each flight leg
# ONDERSTAANDE TWEE FUNCTIES KUNNEN LATER EVT GEMERGED WORDEN
def flight_duration(i,j,k):
    return distance(i, j, R_E=6371) / fleet[0][k] * 60 + 15 + 15 # Round up to the nearest 6-fold
print(flight_duration(1,2,1)) # From Paris to Amsterdam

def get_stages_in_flight(i,j,k):
    return math.ceil((flight_duration(i,j,k)) / 6)

print(get_stages_in_flight(1,2,1))

7865.893334240466
-7865.893334240466
58.113158915127194
10


In [12]:
fleet[7][0]

1875.0

[   0    6   12 ... 7188 7194 7200]


In [14]:
# DEZE CEL KAN LATER MISSCHIEN WEG, MAAR WE ZOUDEN DE STATUS KUNNEN BEPALEN DOOR DUS ALTIJD EEN I EN J MEE TE GEVEN AAN EEN VLIEGTUIG, DUS I NAAR I ALS HET VLIEGTUIG STILSTAAT OP DE HUB EN J NAAR J OP EEN SPOKE, I NAAR J ALS IE ONDERWEG IS
state = ['grounded_hub', 
          'grounded_spoke', 
          'airborne_to_hub', 
          'airborne_to_spoke'] 

# Determine current state (what is the current status of the plane?)
def get_current_state(i, j):
    if i == j:  # grounded
        return state[0] if i == 3 else state[1] # grounded at hub or spoke
    else:  
        return state[2] if j == 3 else state[3] # airborne to hub or to spoke

get_current_state(4,4)

'grounded_spoke'

In [47]:
# Create a list of all possible decisions a plane can take at airport i
def get_candidates_decisions_stage(i,k,s):    
    candidates = []
    hub = 3 # Define Frankfurt as a hub
    if i == hub:                 
        for j in range(len(cities)):                                                        
            if i == j:
                candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,s)})         # grounded at hub
            else:
                candidates.append({"From":cities[i], "to":cities[j],"profit":profit(i,j,k,s-get_stages_in_flight(i,j,k))})  # flight to spoke
        
    else:
        candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,s)})                 # grounded at spoke
        candidates.append({"From":cities[i], "to":cities[hub], "profit":profit(i,hub,k,s-get_stages_in_flight(i,j,k))})     # flight to hub
    return candidates

get_candidates_decisions_stage(3,1,0)

[{'From': 'Frankfurt', 'to': 'London', 'profit': -6092.075614547028},
 {'From': 'Frankfurt', 'to': 'Paris', 'profit': -4649.022216926933},
 {'From': 'Frankfurt', 'to': 'Amsterdam', 'profit': -4070.930110137939},
 {'Grounding at': 'Frankfurt', 'profit': None},
 {'From': 'Frankfurt', 'to': 'Madrid', 'profit': -11473.314679311385},
 {'From': 'Frankfurt', 'to': 'Barcelona', 'profit': -9169.861305782539},
 {'From': 'Frankfurt', 'to': 'Munich', 'profit': -3597.5031941410057},
 {'From': 'Frankfurt', 'to': 'Rome', 'profit': -8219.683075988876},
 {'From': 'Frankfurt', 'to': 'Dublin', 'profit': -9122.750057541216},
 {'From': 'Frankfurt', 'to': 'Stockholm', 'profit': -10077.21672322845},
 {'From': 'Frankfurt', 'to': 'Lisbon', 'profit': -14641.914049443585},
 {'From': 'Frankfurt', 'to': 'Berlin', 'profit': -4527.534139804793},
 {'From': 'Frankfurt', 'to': 'Helsinki', 'profit': -12286.078800802836},
 {'From': 'Frankfurt', 'to': 'Warsaw', 'profit': -7785.63881533592},
 {'From': 'Frankfurt', 'to': 'E

In [48]:
# Create a list of all possible decisions a plane can take at airport i
def get_candidates_decisions_time(i,k,t):    
    candidates = []
    hub = 3 # Define Frankfurt as a hub
    if i == hub:                 
        for j in range(len(cities)):                                                        
            if i == j:
                candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,t)})         # grounded at hub
            else:
                candidates.append({"From":cities[i], "to":cities[j],"profit":profit(i,j,k,t)})  # flight to spoke
        
    else:
        candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,t)})                 # grounded at spoke
        candidates.append({"From":cities[i], "to":cities[hub], "profit":profit(i,hub,k,t)})     # flight to hub
    return candidates

get_candidates_decisions_time(3,1,0)

[{'From': 'Frankfurt', 'to': 'London', 'profit': -6092.075614547028},
 {'From': 'Frankfurt', 'to': 'Paris', 'profit': -4649.022216926933},
 {'From': 'Frankfurt', 'to': 'Amsterdam', 'profit': -4070.930110137939},
 {'Grounding at': 'Frankfurt', 'profit': None},
 {'From': 'Frankfurt', 'to': 'Madrid', 'profit': -11473.314679311385},
 {'From': 'Frankfurt', 'to': 'Barcelona', 'profit': -9169.861305782539},
 {'From': 'Frankfurt', 'to': 'Munich', 'profit': -3597.5031941410057},
 {'From': 'Frankfurt', 'to': 'Rome', 'profit': -8219.683075988876},
 {'From': 'Frankfurt', 'to': 'Dublin', 'profit': -9122.750057541216},
 {'From': 'Frankfurt', 'to': 'Stockholm', 'profit': -10077.21672322845},
 {'From': 'Frankfurt', 'to': 'Lisbon', 'profit': -14641.914049443585},
 {'From': 'Frankfurt', 'to': 'Berlin', 'profit': -4527.534139804793},
 {'From': 'Frankfurt', 'to': 'Helsinki', 'profit': -12286.078800802836},
 {'From': 'Frankfurt', 'to': 'Warsaw', 'profit': -7785.63881533592},
 {'From': 'Frankfurt', 'to': 'E

In [16]:
# Pick the list item with the greatest profit
def optimal_decision(i,k,t):
    return max(
        get_candidates_decisions(i,k,t),
        key = lambda x:x['profit'] if x['profit'] is not None else float('-inf')
        )

print(f"The most profitable decision is: {optimal_decision(3,1,5)}")

The most profitable decision is: {'From': 'Frankfurt', 'to': 'Madrid', 'profit': 1467.3455272095653}


40.03333333333333


### Helper Functions

In [18]:
def calculate_revenue(route, flow):                                                                           # Bij ons binnenin def profit(i,j,k,t)
    """Calculate revenue for a given route and cargo flow."""
    distance = FLIGHT_DATA[route]["distance"]
    yield_per_rtk = 0.26  # Revenue per RTK
    return yield_per_rtk * distance * flow

def calculate_cost(route, aircraft_type):                                                                     # Bij ons binnenin def profit(i,j,k,t)
    """Calculate operating costs for a given route and aircraft."""
    flight_data = FLIGHT_DATA[route]
    time_cost = (flight_data["distance"] / aircraft_type["speed"]) * aircraft_type["time_cost"]
    fuel_cost = (flight_data["distance"] ** 1.5) * aircraft_type["fuel_cost"]
    fixed_cost = aircraft_type["fixed_cost"]
    return time_cost + fuel_cost + fixed_cost

def transition(state, decision):                                                                              
    """Compute the next state and associated cost/reward."""
    t, airport, aircraft = state
    route = decision["route"]
    if route is None:  # No flight decision
        return (t + TIME_STEP, airport, aircraft), 0
    next_airport = route["destination"]
    flight_time = route["flight_time"]
    if t + flight_time > TOTAL_TIME:
        return None, 0  # Invalid transition
    revenue = calculate_revenue(route, decision["cargo"])
    cost = calculate_cost(route, aircraft)
    return (t + flight_time, next_airport, aircraft), revenue - cost

def dp_solve():
    """Dynamic programming solver."""
    for t in reversed(time_stages):
        for airport in AIRPORTS:
            for aircraft in AIRCRAFT_TYPES:
                state = (t, airport, aircraft)
                dp_table[state] = -np.inf  # Initialize with negative infinity
                best_decision = None
                # Evaluate decisions
                for decision in get_possible_decisions(state):                                          # Dit deel is bij ons nu gedekt door def optimal_decision(i,k,t)
                    next_state, reward = transition(state, decision)
                    if next_state is not None:
                        value = reward + dp_table.get(next_state, 0)
                        if value > dp_table[state]:
                            dp_table[state] = value
                            best_decision = decision
                # Store best decision for traceback
                dp_table[state] = {"value": dp_table[state], "decision": best_decision}

def get_possible_decisions(state):                                                                      # Bij ons def get_candidate_decisions
    """Generate possible decisions for a given state."""
    t, airport, aircraft = state
    decisions = []
    # Add all possible flights
    for route in FLIGHT_DATA:
        if route["origin"] == airport:
            # Check constraints (range, runway, etc.)
            decisions.append({"route": route, "cargo": min(route["demand"], MAX_CAPACITY[aircraft])})   # In onze functie is nog geen cargo meegenomen
    # Add idle (no flight) decision
    decisions.append({"route": None, "cargo": 0})
    return decisions

def traceback():
    """Traceback to reconstruct the optimal solution."""
    schedule = []
    state = (0, HUB, AIRCRAFT_TYPES[0])  # Starting state
    while state in dp_table:
        best_decision = dp_table[state]["decision"]
        if best_decision["route"] is not None:
            schedule.append({"state": state, "decision": best_decision})
        next_state, _ = transition(state, best_decision)
        state = next_state
    return schedule

# Main Execution
def main():
    # Load data from Excel
    # AIRCRAFT_TYPES = pd.read_excel("FleetType.xlsx")
    # AIRPORTS = pd.read_excel("AirportData.xlsx")
    # DEMAND = pd.read_excel("GroupX.xlsx")
    # Preprocess FLIGHT_DATA and DEMAND
    # TODO

    # Solve DP
    dp_solve()

    # Traceback to get the optimal solution
    optimal_schedule = traceback()

    # Print or save results
    print("Optimal Schedule:")
    for entry in optimal_schedule:
        print(entry)

if __name__ == "__main__":
    main()

NameError: name 'time_stages' is not defined